
# Import Data

Importing AXP data.

In [1]:

import pandas as pd

amex_df = pd.read_csv('../../data/gen/usAXP_df.csv',
                           parse_dates=True,
                           index_col=0)
# Strip whitespace from column names
amex_df.columns = amex_df.columns.str.strip()
# Print
amex_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key
,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4



# Even Faster Approach: Using Numba

Optimize Calculation using Numba

## Processing Data

Adding slope & velocity to each day.


In [2]:

import numpy as np

extra_df = amex_df.copy()
extra_df['slope'] = pd.Series(np.gradient(extra_df.close), extra_df.index, name='slope')
extra_df['velocity'] = pd.Series(np.gradient(extra_df.slope), extra_df.index, name='velocity')
# Print
extra_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key,slope,velocity
,,,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0,0.420,-0.0850
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1,0.335,-0.1825
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2,0.055,-0.0150
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3,0.305,0.2675
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4,0.590,0.1125



## Performing Backtests

1. Prepare Parameters
1. Dfine a DataFrame to handle results
1. For Loops
1. Prepare Maps & Filters

## Preparing Parameters


In [38]:

import itertools

buy_slope_threshold_list = np.arange(-0.5, 0.5, 0.1)
buy_velocity_threshold_list = np.arange(-0.5, 0.5, 0.1)
close_slope_threshold_list = np.arange(-0.2, 0.2, 0.1)
close_velocity_threshold_list = np.arange(-0.2, 0.2, 0.1)
sell_slope_threshold_list = np.arange(-0.5, 0.5, 0.1)
sell_velocity_threshold_list = np.arange(-0.5, 0.5, 0.1)

task_list = list(itertools.product(
    buy_slope_threshold_list, 
    buy_velocity_threshold_list, 
    close_slope_threshold_list, 
    close_velocity_threshold_list, 
    sell_slope_threshold_list, 
    sell_velocity_threshold_list
))
print("Params Ready, {} tasks to run. ".format(len(task_list)))


Params Ready, 160000 tasks to run. 



## For Loops


In [ ]:

from tqdm import tqdm
from datetime import datetime
from numba import jit 
import numba as nb

startTime = datetime.now()

@jit 
def calc_long_profit(array):
    array = array / 100 + 1
    return array.prod() - 1
@jit 
def calc_short_profit(array):
    array = array / 100 - 1
    return array.prod() - 1
    
def back_test():
    
    # Defining a DataFrame to handle results
    final_result = pd.DataFrame(columns=['buy_slope_threshold', 'buy_velocity_threshold', 'close_slope_threshold', 'close_velocity_threshold', 'sell_slope_threshold', 'sell_velocity_threshold', 'profit_percentage'])
    
    for buy_slope_threshold, \
        buy_velocity_threshold, \
        close_slope_threshold, \
        close_velocity_threshold, \
        sell_slope_threshold, \
        sell_velocity_threshold in tqdm(task_list):
        
        # Prepare Maps & Filters 
        
        long_mask = (extra_df.slope > buy_slope_threshold) & (extra_df.velocity > buy_velocity_threshold)
        long_close = (extra_df.slope < -close_slope_threshold) & (extra_df.velocity < -close_velocity_threshold)
        long_series = extra_df.p_change[long_mask & long_close==False]
        long_array = long_series.values 
        long_profit = np.abs(np.product(long_array/100 + 1)) - 1
        # long_profit = calc_long_profit(long_array)
        
        short_mask = (extra_df.slope < sell_slope_threshold) & (extra_df.velocity < sell_velocity_threshold)
        short_close = (extra_df.slope > close_slope_threshold) & (extra_df.velocity > close_velocity_threshold)
        short_series = extra_df.p_change[short_mask & short_close==False]
        short_array = short_series.values
        short_profit = np.abs(np.product(short_array/100 - 1)) - 1
        # short_profit = calc_short_profit(short_array)
    
        trade_profit = long_profit + short_profit
    
        result_row = [buy_slope_threshold, 
                      buy_velocity_threshold, 
                      close_slope_threshold, 
                      close_velocity_threshold, 
                      sell_slope_threshold, 
                      sell_velocity_threshold, 
                      trade_profit * 100]
    
        final_result = final_result.append(pd.Series(result_row, index=final_result.columns), ignore_index=True)
    
    return final_result

back_test_jit = nb.jit(back_test)
final_result = back_test_jit()

print("Finished {} tasks in {}".format(len(task_list), datetime.now() - startTime))



<ipython-input-50-0400cdcf818a>:18: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "back_test" failed type inference due to: Untyped global name 'tqdm': cannot determine Numba type of <class 'type'>

File "<ipython-input-50-0400cdcf818a>", line 28:
def back_test():
    <source elided>
        sell_slope_threshold, \
        sell_velocity_threshold in tqdm(task_list):
        ^

  def back_test():
<ipython-input-50-0400cdcf818a>:18: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "back_test" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-50-0400cdcf818a>", line 23:
def back_test():
    <source elided>
    
    for buy_slope_threshold, \
    ^

  def back_test():
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numba/compiler.py:725: NumbaWarning: Function "back_test" was compiled


# Analysing Results


In [ ]:
        
final_result = final_result.sort_values(by=['profit_percentage'],
                                        ascending=False)
final_result.head()
